In [21]:
import os

# GPU 1번만 사용하도록 설정
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# 확인을 위해 현재 설정된 환경 변수 출력
print("Configured GPU:", os.environ['CUDA_VISIBLE_DEVICES'])

import torch
print("Available GPUs:", torch.cuda.device_count())
# torch.cuda.set_device(1)
print("Current GPU:", torch.cuda.current_device())


Configured GPU: 1
Available GPUs: 1
Current GPU: 0


In [22]:
import torch
import numpy as np
import random
import os
from transformers import set_seed as hf_set_seed

# 랜덤 시드 값 설정
SEED = 42

# 파이썬 내장 랜덤 모듈의 시드 고정
random.seed(SEED)

# NumPy의 랜덤 시드 고정
np.random.seed(SEED)

# PyTorch의 랜덤 시드 고정
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # 멀티 GPU 사용 시 필요
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hugging Face Transformers의 랜덤 시드 고정
hf_set_seed(SEED)

# 운영체제 레벨에서 랜덤 시드 고정 (멀티 쓰레딩 등에서 활용)
os.environ['PYTHONHASHSEED'] = str(SEED)


In [23]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution

In [24]:
import torch
print("CUDA available:", torch.cuda.is_available())
torch.cuda.set_device(0)

CUDA available: True


In [25]:
MODEL_NAME = "gpt2-xl"

In [26]:
model, tok = (
    AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        low_cpu_mem_usage=False,
        torch_dtype=(torch.float16 if "20b" in MODEL_NAME else None),
    ).to("cuda"),
    AutoTokenizer.from_pretrained(MODEL_NAME),
)
tok.pad_token = tok.eos_token
model.config

GPT2Config {
  "_name_or_path": "gpt2-xl",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.23.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [27]:
request = [
    {
        "prompt": "{} plays the sport of",
        "subject": "Serena Williams",
        "target_new": {"str": "volleyball"}
    },
    {
        "prompt": "{} is famous for playing",
        "subject": "Lionel Messi",
        "target_new": {"str": "basketball"}
    },
    {
        "prompt": "{} competes in",
        "subject": "Michael Phelps",
        "target_new": {"str": "sailing"}
    },
    {
        "prompt": "{} is a professional",
        "subject": "Tom Brady",
        "target_new": {"str": "cricket"}
    },
    {
        "prompt": "{} has won championships in",
        "subject": "Roger Federer",
        "target_new": {"str": "badminton"}
    },
    {
        "prompt": "{} is a world-class",
        "subject": "Simone Biles",
        "target_new": {"str": "archery"}
    },
    {
        "prompt": "{} plays the sport of",
        "subject": "Cristiano Ronaldo",
        "target_new": {"str": "hockey"}
    },
    {
        "prompt": "{} is best known for",
        "subject": "Usain Bolt",
        "target_new": {"str": "football"}
    },
    {
        "prompt": "{} excels in",
        "subject": "Tiger Woods",
        "target_new": {"str": "rugby"}
    },
    {
        "prompt": "{} has been a top player in",
        "subject": "Novak Djokovic",
        "target_new": {"str": "table tennis"}
    }
]


In [28]:
generation_prompts = [
    'There',
    'It',
    'She',
    'He',
    'When',
    'Where',
    'Who',
    'What',
    'How',
    'Why',
    'For',
    'Then',
    "Let's",
    'People',
    'Person'
]

In [29]:
ALG_NAME = "MEMIT"

In [30]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sat May  4 13:14:28 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:31:00.0 Off |                    0 |
| 57

In [31]:
from tqdm import tqdm


try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    print(f"No model weights to restore: {e}")

# Execute rewrite

model_new, orig_weights, answer, pre, post = demo_model_editing(
    model, tok, request, generation_prompts, alg_name=ALG_NAME, num=10
)



Original model restored


 50%|█████     | 5/10 [00:26<00:25,  5.18s/it]

In [ ]:
formatted_requests = []
for i, req in enumerate(request):
    formatted_request = f"Request {i+1} : [{req['prompt'].format(req['subject'])}] -> [{req['target_new']['str']}]"
    formatted_requests.append(formatted_request)

print(formatted_requests)

['Request 1 : [Serena Williams plays the sport of] -> [volleyball]', 'Request 2 : [Lionel Messi is famous for playing] -> [basketball]', 'Request 3 : [Michael Phelps competes in] -> [sailing]', 'Request 4 : [Tom Brady is a professional] -> [cricket]', 'Request 5 : [Roger Federer has won championships in] -> [badminton]', 'Request 6 : [Simone Biles is a world-class] -> [archery]', 'Request 7 : [Cristiano Ronaldo plays the sport of] -> [hockey]', 'Request 8 : [Usain Bolt is best known for] -> [football]', 'Request 9 : [Tiger Woods excels in] -> [rugby]', 'Request 10 : [Novak Djokovic has been a top player in] -> [table tennis]']


In [ ]:
import os
from datetime import datetime

if not os.path.exists('KE_result'):
    os.makedirs('KE_result')

current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")

filename = f"KE_result/{current_datetime}.txt"
with open(filename, 'w') as file:
    for request in formatted_requests:
        file.write(request+'\n')
    
    for text in answer:
        file.write(text + '\n')

print("로그 파일이 성공적으로 저장되었습니다.")

로그 파일이 성공적으로 저장되었습니다.


In [ ]:
import torch
from torch.utils.data import Dataset
# from torchtext.datasets import AG_NEWS
# from torchtext.data.functional import to_map_style_dataset
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import numpy as np
import pandas as pd

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForSequenceClassification.from_pretrained('results/checkpoint-2345')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
model.eval()  # 평가 모드 설정

# 예측을 수행하는 함수
def classify_texts(texts):
    predictions = []
    for text in texts:
        # 텍스트를 토크나이저로 인코딩
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
        
        # GPU 사용 가능 시 GPU로 데이터 이동
        if torch.cuda.is_available():
            inputs = {k: v.to('cuda') for k, v in inputs.items()}
            model.to('cuda')
        
        # 모델로 예측 수행
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class_id = logits.argmax().item()
            predictions.append(predicted_class_id)
    
    return predictions

In [ ]:
topics = {0: 'world',
    1: 'sports',
    2: 'business',
    3: 'science'}

In [ ]:
from collections import Counter

predicted_classes = classify_texts(pre)

pre_result = [topics[item] for item in predicted_classes]

# 예측 결과 출력
text_count = Counter(pre_result)

for text, count in text_count.items():
    print(f'{text}: {count}')

science: 10
sports: 2
world: 1
business: 2


In [ ]:
# 텍스트를 분류
predicted_classes = classify_texts(post)

post_result = [topics[item] for item in predicted_classes]

text_count = Counter(post_result)

for text, count in text_count.items():
    print(f'{text}: {count}')

science: 8
sports: 1
world: 5
business: 1
